In [1]:
from ast import literal_eval
import os

import pandas as pd

from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

# Prepare data and split it

In [2]:
#os.makedirs("./spacy")

In [3]:
train = pd.read_excel("../../data/baseline/train.xlsx", engine="openpyxl")
val = pd.read_excel("../../data/baseline/val.xlsx", engine="openpyxl")
test = pd.read_excel("../../data/baseline/test.xlsx", engine="openpyxl")

In [4]:
training_data = []

for index, row in tqdm(train.iterrows(), total=len(train)):
    line = row['line_clean']
    ners = literal_eval(row['labels'])
    training_data.append((line, ners))

100%|██████████| 11233/11233 [00:00<00:00, 18447.73it/s]


In [5]:
valid_data = []

for index, row in tqdm(val.iterrows(), total=len(val)):
    line = row['line_clean']
    ners = literal_eval(row['labels'])
    valid_data.append((line, ners))

100%|██████████| 3009/3009 [00:00<00:00, 20948.58it/s]


In [6]:
test_data = []

for index, row in tqdm(test.iterrows(), total=len(test)):
    line = row['line_clean']
    ners = literal_eval(row['labels'])
    test_data.append((line, ners))

100%|██████████| 3441/3441 [00:00<00:00, 20039.54it/s]


In [7]:


nlp = spacy.blank("ru")

# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
    if type(text) == str:
        doc = nlp(text)
        ents = []
        if len(annotations):
            for start, end, label in annotations:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
        doc.ents = ents
        db.add(doc)
    else:
        print(text, annotations)
db.to_disk("./spacy/train.spacy")

nan []


In [8]:
db = DocBin()
for text, annotations in valid_data:
    if type(text) == str:
        doc = nlp(text)
        ents = []
        if len(annotations):
            for start, end, label in annotations:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
        doc.ents = ents
        db.add(doc)
    else:
        print(text, annotations)
db.to_disk("./spacy/valid.spacy")

In [9]:
db = DocBin()
for text, annotations in test_data:
    if type(text) == str:
        doc = nlp(text)
        ents = []
        if len(annotations):
            for start, end, label in annotations:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
        doc.ents = ents
        db.add(doc)
    else:
        print(text, annotations)
db.to_disk("./spacy/test.spacy")

# Create config

In [ ]:
%%bash

python -m spacy init fill-config base_config.cfg config.cfg

# Run training (better in comand line)

In [ ]:
%%bash

python -m spacy train config.cfg --gpu-id 0 --output ./output --paths.train ./spacy/train.spacy --paths.dev ./spacy/valid.spacy